<a href="https://colab.research.google.com/github/syaefulalam/sentimen-analysis/blob/master/top%20modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Gensim for Topic Modeling**

We’re going to use the gensim implementations because they offer more functionality out of the box and then we’ll replicate that functionality with sklearn. Let’s first prepare the dataset we’ll be working with.

In [1]:
!pip install sastrawi
!pip install pyldavis
!pip install gensim==3.8.0

import nltk
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re 

import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 215kB 4.9MB/s 
     |████████████████████████████████| 1.6MB 4.7MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=92ee1491636332d404e2bde007deb11ed1913b2e17b60e43ad009aa3d6579bfa
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis
     |████████████████████████████████| 24.2MB 85.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!pip install gensim==3.8.0
import pkg_resources
pkg_resources.get_distribution("gensim").version


'3.6.0'

In [33]:
from google.colab import files
uploaded = files.upload()

Saving berita2.txt to berita2.txt


In [34]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

User uploaded file "berita2.txt" with length 77108 bytes


In [0]:
import pandas as pd
import io

In [40]:
article = open('berita2.txt','r').read().split('BERHENTI DISINI')
len(article)

32

In [41]:
article_clean = []
for text in article:
    text = BeautifulSoup(text, 'html.parser').getText()
    article_clean.append(text)
article = article_clean
print(article[0][:100])

﻿

Kroasia: Melawan Argentina adalah Pertandingan Termudah











Jakarta, CNN Indonesia -- Agun


Tokenize and clean stopwords

In [0]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [0]:
def tokenize_and_stem(text):
    stopwords = nltk.corpus.stopwords.words('indonesian')
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token) and token not in stopwords:
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [44]:
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in article:
    tokenized_data.append(tokenize_and_stem(text))

# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...


[(2, 1), (3, 1), (11, 1), (19, 4), (22, 2), (24, 1), (26, 1), (44, 1), (50, 1), (164, 2), (183, 9), (196, 1), (223, 2), (230, 1), (252, 4), (274, 1), (280, 1), (284, 1), (309, 1), (314, 1), (335, 2), (341, 1), (404, 1), (431, 1), (434, 1), (452, 1), (465, 1), (474, 1), (480, 1), (485, 2), (489, 1), (500, 1), (504, 1), (520, 1), (523, 1), (534, 1), (538, 2), (546, 1), (547, 9), (552, 1), (596, 1), (597, 3), (599, 1), (619, 2), (676, 1), (845, 1), (904, 1), (914, 2), (927, 1), (932, 1), (967, 1), (1032, 1), (1038, 2), (1165, 2), (1174, 1), (1283, 1), (1397, 1), (1398, 8), (1399, 1), (1400, 1), (1401, 3), (1402, 1), (1403, 1), (1404, 2), (1405, 4), (1406, 1), (1407, 2), (1408, 3), (1409, 1), (1410, 1), (1411, 1), (1412, 3), (1413, 1), (1414, 4), (1415, 2), (1416, 1), (1417, 1), (1418, 1), (1419, 1), (1420, 1), (1421, 1), (1422, 1), (1423, 1), (1424, 2), (1425, 1), (1426, 1), (1427, 1), (1428, 1), (1429, 1), (1430, 1), (1431, 4), (1432, 1), (1433, 1), (1434, 1), (1435, 1), (1436, 1), (1437

In [0]:
NUM_TOPICS = 4

# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha = 'auto', eval_every=5)#, per_word_topics=True)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

We’re going to run LDA and LSI (Latent Semantic Indexing AKA Latent Semantic Analysis) models, which implementation included in the gensim package.

Let’s now display the topics the two models have inferred:

In [46]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.024*"persen" + 0.007*"indeks" + 0.007*"diskon" + 0.007*"jakarta" + 0.006*"lemah" + 0.006*"indonesia" + 0.005*"laku" + 0.005*"saham" + 0.005*"cnn" + 0.004*"oknum"
Topic #1: 0.009*"persen" + 0.008*"main" + 0.007*"dunia" + 0.006*"menang" + 0.005*"indonesia" + 0.005*"golf" + 0.005*"piala" + 0.005*"rusia" + 0.005*"jakarta" + 0.005*"saya"
Topic #2: 0.007*"polisi" + 0.006*"duga" + 0.006*"jakarta" + 0.006*"main" + 0.006*"novel" + 0.006*"indonesia" + 0.005*"dunia" + 0.005*"tanding" + 0.004*"salah" + 0.004*"argentina"
Topic #3: 0.019*"persen" + 0.006*"indonesia" + 0.006*"lapor" + 0.006*"uang" + 0.005*"dolar" + 0.005*"hasil" + 0.005*"balap" + 0.005*"belanja" + 0.005*"duga" + 0.005*"jakarta"
LSI Model:
Topic #0: -0.731*"persen" + -0.216*"lemah" + -0.171*"dolar" + -0.165*"minus" + -0.133*"indeks" + -0.128*"bunga" + -0.116*"kuat" + -0.111*"uang" + -0.104*"as" + -0.103*"dagang"
Topic #1: 0.385*"novel" + 0.311*"oknum" + 0.300*"duga" + 0.271*"jenderal" + 0.165*"kpk" + 0.142*"main

Let’s now put the models to work and transform unseen documents to their topic distribution:

Unsupported Cell Type. Double-Click to inspect/edit the content.

In [47]:
text = "Pertandingan berjalan dengan seru. Tim lawan berhasil dikalahkan dengan skor 1-0."
bow = dictionary.doc2bow(tokenize_and_stem(text))

print(lda_model[bow]) 
print(lsi_model[bow])
print(bow)

[(0, 0.06474505), (1, 0.13383561), (2, 0.7391487), (3, 0.062270623)]
[(0, -0.13315407974835303), (1, 0.35909145512635154), (2, -0.8065685310165075), (3, -0.03966512669010286)]
[(19, 1), (46, 1), (75, 1), (137, 1), (404, 1), (454, 1), (930, 1)]


The LDA result can be interpreted as a distribution over topics. Gensim offers a simple way of performing similarity queries using topic models.

In [48]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(article[document_id][:1000])

[(2, 0.97475624), (9, 0.97475624), (28, 0.9207067), (31, 0.6229303), (15, 0.52913433), (0, 0.18847016), (4, 0.18847016), (5, 0.18847016), (11, 0.18847016), (18, 0.18847016)]









Jakarta, CNN Indonesia -- Kapolres Jakarta Selatan, Komisaris Besar Indra Jafar mengatakan pihaknya bakal menindaklanjuti laporan seorang warga, Ronny Yuniarto terkait kasus pemukulan dan pengeroyokan yang diduga dilakukan oleh Politisi PDIP Herman Hery.





Indra mengatakan pihaknya masih mengumpulkan keterangan polisi lalu lintas (Polantas) yang menjadi saksi di lokasi kejadian serta hasil visum dari korban untuk menindaklanjuti laporan tersebut.





"Proses tetap kita lanjutkan, kita masih minta hasil visum rerhadap korban, kita minta di salah satu rumah sakit rujukan, selain itu juga masih dilakukan penyelidikan yang lain untuk mencari saksi-saksi," ujar Indra kepada wartawan di Mapolres Jakarta Selatan, Kamis (21/6).





Indra juga mengatakan bakal mendalami keterangan saksi dari polisi yang dinila

Notice how the factors corresponding to each component (topic) add up to 1. That’s not a coincidence. Indeed, LDA considers documents as being generated by a mixture of the topics. The purpose of LDA is to compute how much of the document was generated by which topic.

LDA is an iterative algorithm. Here are the two main steps:

In the initialization stage, each word is assigned to a random topic.
Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration:
What’s the probability of the word belonging to a topic
What’s the probability of the document to be generated by a topic
Due to these important qualities, we can visualize LDA results easily. We’re going to use a specialized tool called PyLDAVis:

In [49]:
import pyLDAvis.gensim
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
panel

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.042376 -0.021341       1        1  38.757179
3      0.011717  0.039274       2        1  25.266870
1     -0.020717 -0.012197       3        1  21.955357
2     -0.033376 -0.005736       4        1  14.020589, topic_info=     Category        Freq       Term       Total  loglift  logprob
471   Default   24.000000       main   24.000000  30.0000  30.0000
180   Default  105.000000     persen  105.000000  29.0000  29.0000
1398  Default   14.000000      balap   14.000000  28.0000  28.0000
45    Default   18.000000      lapor   18.000000  27.0000  27.0000
835   Default   18.000000      novel   18.000000  26.0000  26.0000
6     Default   11.000000  argentina   11.000000  25.0000  25.0000
55    Default   19.000000     polisi   19.000000  24.0000  24.0000
133   Default   21.000000     indeks   21.000000  23.0000  23.0000
474   Default   14.000000     menang   14.000000  22.0000  22.0000
75    Default   18.000000    tanding   18.000000  21.0000  21.0000
1447  Default   10.000000     vettel   10.000000  20.0000  20.0000
767   Default   13.000000       aman   13.000000  19.0000  19.0000
42    Default    8.000000    kroasia    8.000000  18.0000  18.0000
437   Default   28.000000      dunia   28.000000  17.0000  17.0000
1803  Default   14.000000    belanja   14.000000  16.0000  16.0000
309   Default   13.000000      salah   13.000000  15.0000  15.0000
254   Default   26.000000       duga   26.000000  14.0000  14.0000
946   Default   14.000000       golf   14.000000  13.0000  13.0000
46    Default   17.000000      lawan   17.000000  12.0000  12.0000
1777  Default    6.000000      messi    6.000000  11.0000  11.0000
492   Default   13.000000      rusia   13.000000  10.0000  10.0000
266   Default    6.000000      indra    6.000000   9.0000   9.0000
730   Default    7.000000      mudik    7.000000   8.0000   8.0000
520   Default   13.000000       baik   13.000000   7.0000   7.0000
278   Default   12.000000     korban   12.000000   6.0000   6.0000
483   Default   16.000000      piala   16.000000   5.0000   5.0000
802   Default   15.000000   jenderal   15.000000   4.0000   4.0000
1476  Default   23.000000     diskon   23.000000   3.0000   3.0000
19    Default   18.000000      hasil   18.000000   2.0000   2.0000
262   Default    4.000000     herman    4.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
300    Topic4    1.459117      pukul    4.151897   0.9189  -6.4147
55     Topic4    5.866191     polisi   19.616850   0.7575  -5.0233
835    Topic4    5.385378      novel   18.151737   0.7496  -5.1088
309    Topic4    3.948663      salah   13.316858   0.7490  -5.4191
767    Topic4    3.821413       aman   13.364563   0.7127  -5.4519
252    Topic4    1.999580      depan    6.341283   0.8105  -6.0996
278    Topic4    3.531608     korban   12.356049   0.7123  -5.5308
75     Topic4    4.704544    tanding   18.651991   0.5872  -5.2440
1791   Topic4    1.163072   sampaoli    3.500280   0.8629  -6.6414
471    Topic4    5.459287       main   24.460718   0.4649  -5.0952
254    Topic4    5.648079       duga   26.618519   0.4144  -5.0612
264    Topic4    2.734898      hukum   10.627949   0.6073  -5.7864
1398   Topic4    3.404652      balap   14.192402   0.5371  -5.5674
454    Topic4    2.712165      kalah   10.761648   0.5864  -5.7948
802    Topic4    3.518128   jenderal   15.559322   0.4779  -5.5346
46     Topic4    3.720195      lawan   17.443827   0.4194  -5.4787
275    Topic4    2.372642    keroyok    9.451571   0.5825  -5.9285
45     Topic4    3.727209      lapor   18.510113   0.3620  -5.4769
26     Topic4    5.594629    jakarta   36.332359   0.0937  -5.0707
437    Topic4    4.730576      dunia   28.932831   0.1537  -5.2385
34     Topic4    2.711912       kami   12.209455   0.4601  -5.7949
24     Topic4    4.992807  indonesia   36.807800  -0.0331  -5.1845
839    Topic4  